This is the final project "sandbox" for CSC6621 - Arsalon's Version


In [ ]:
# imports 
import os
import subprocess
import shutil
import py7zr

# not required if using the Docker Image 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # type: ignore
import numpy as np

In [2]:
from pyunpack import Archive

# Define the path to the directory containing the split archive
archive_directory = 'data/img_celeba.7z'

# Specify the first file of the split archive
first_part_path = os.path.join(archive_directory, 'img_celeba.7z.001')

# Define the output directory for the extracted files
output_directory = 'data/img_celeba_extracted'

# Extract the archive
try:
    Archive(first_part_path).extractall(output_directory)
    print("Extraction successful.")
except Exception as e:
    print("Failed to extract:", e)



Extraction successful.


In [ ]:
# create train test subfolders

# Path to the directory where all images are stored
image_directory = os.path.join(output_directory, 'img_celeba')

# Create directories for train and test datasets if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get all file names in the output directory
all_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# Split files into train and test sets (80-20 split here)
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# Function to copy files to new directories
def copy_files(files, directory):
    for file in files:
        shutil.copy(file, directory)

# Copy files to respective directories
copy_files(train_files, train_dir)
copy_files(test_files, test_dir)

print("Files successfully split into training and testing directories.")


ImageDataGenerator: 

-From Tensorflow helps performing operations such as rescaling, rotating, shifting, shearing, and flipping. 
-Reads images in large numbers from a directory and automatically feed them to the model in batches

In [ ]:
# Create a Train-Test Split

train_dir = os.path.join(output_directory, 'train')
test_dir = os.path.join(output_directory, 'test')

# Create data generators - allows us to feed images to the model in batches
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Use 20% of training data for validation
test_datagen = ImageDataGenerator(rescale=1./255) # rescale from (0,255) to (0,1)

# Train  Generator 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Depending on your CNN input layer
    batch_size=32,
    class_mode='categorical',  
    subset='training'  # Set as training data
)

# Test Generator 
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
# simple CNN for training

from keras.models import Sequential # type: ignore
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use 'softmax' for categorical
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
